# 第 11 章: 運用管理 - データライフサイクル管理

本ノートブックでは、「データライフサイクル管理」節で紹介されている、スナップショットの期限設定と期限に基づく削除を実行することができます。具体的には、以下の内容を実行できます:

* スナップショット保持期間とスナップショット最小保持数の設定と利用
* データ削除要件にもとづいて削除する
* ブランチやタグによる詳細なデータの保持管理

なお各例における保持期間については、テスト用のため数分など短い時間に設定しています。実運用で利用する際は、会社のポリシーなどに合わせた期間を設定してください。

## 事前準備

In [ ]:
import time
import pyspark
from pyspark.sql import SparkSession
CATALOG = "my_catalog"
CATALOG_URL = "http://server:8181/"
S3_ENDPOINT = "http://minio:9000"
SPARK_VERSION = pyspark.__version__
SPARK_MINOR_VERSION = '.'.join(SPARK_VERSION.split('.')[:2])
ICEBERG_VERSION = "1.8.1"

In [ ]:
spark = (
    SparkSession.builder
        .config("spark.jars.packages", f"org.apache.iceberg:iceberg-spark-runtime-{SPARK_MINOR_VERSION}_2.12:{ICEBERG_VERSION},org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VERSION}")
        .config(f"spark.sql.catalog.{CATALOG}", "org.apache.iceberg.spark.SparkCatalog")
        .config(f"spark.sql.catalog.{CATALOG}.type", "rest")
        .config(f"spark.sql.catalog.{CATALOG}.uri", CATALOG_URL)
        .config(f"spark.sql.catalog.{CATALOG}.s3.endpoint", S3_ENDPOINT)
        .config(f"spark.sql.catalog.{CATALOG}.view-endpoints-supported", "true")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.defaultCatalog", "my_catalog")
        .getOrCreate()
)

In [ ]:
%sql spark

### (Optional) データベースの作成
データベースを作成していない場合、以下のセルを実行してください。既にデータベースが存在する場合は、本ステップにつきましてはスキップしてください。

In [ ]:
%%sql
CREATE DATABASE IF NOT EXISTS db

## スナップショット保持期間とスナップショット最小保持数の設定と利用
まずは、データ保持期間を設定するための基本テーブルプロパティである `history.expire.max-snapshot-age-ms` (デフォルト: 5 日間)と `history.expire.min-snapshots-to-keep` (デフォルト: 1) の設定方法について紹介します。またこれらの設定に基づいて、データを削除するための Spark プロシージャ `expire_snapshots` の実行方法についても確認します。
なお、本ノートブックではテスト目的で、スナップショットの保持期間を 1 分に設定しています。実際のユースケースでは、会社のポリシーなどに従って期限を設定します。

### スナップショット保持期間を設定する

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg_practice (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg
TBLPROPERTIES ('history.expire.max-snapshot-age-ms'='60000') -- 1 min に設定

In [ ]:
records = [
    "('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink')",
    "('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink')",
    "('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink')",
    "('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery')",
    "('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')"
]

for record in records:
    spark.sql(f"INSERT INTO db.sales_iceberg_practice VALUES {record}")
    time.sleep(20) # 20 秒ごとにテーブルに書き込む

以下のメタデータクエリを実行し、この時点における Iceberg テーブルの各スナップショットの作成時間とスナップショット数を確認します。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_practice.history

`expire_snapshots` プロシージャを実行し、保持期間を超過したスナップショット (今回は**作成後から 1 分**を超過したもの)を削除します。

In [ ]:
%%sql
CALL system.expire_snapshots (
    table => 'db.sales_iceberg_practice'
)

以下のメタデータクエリを実行し、先ほどのメタデータクエリと比較してみましょう。スナップショット作成から 1 分間が経過したものが全て削除されることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_practice.history

### スナップショット最小保持数を設定する

次に、`history.expire.min-snapshots-to-keep` (デフォルト: 1)を変更し、スナップショット保持期間を超えた場合でも、スナップショットが保持されることを確認してみましょう。まずは保持期間を 5 に設定します。

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg_practice SET TBLPROPERTIES ('history.expire.min-snapshots-to-keep'='5')

追加で 2 レコード書き込みます。ここでは、各書き込みで作成されるスナップショットがその保持期間を超過するように 1 分 10 秒待つようにしています。

In [ ]:
records = [
    "('mocha', 4.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2013-11-26T12:49:43Z', 'drink')",
    "('egg', 1.00, 3176, 'GRVQARCD6COVQARCD6CO', TIMESTAMP '2025-02-10 11:15:31', 'grocery')",
]

for record in records:
    spark.sql(f"INSERT INTO db.sales_iceberg_practice VALUES {record}")
    time.sleep(70) # 70 秒ごとにテーブルに書き込む

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_practice.history

`expire_snapshots` をもう一度実行して、保持期間を超過したスナップショットが保持されるか確認しましょう。

In [ ]:
%%sql
CALL system.expire_snapshots (
    table => 'db.sales_iceberg_practice'
)

以下のメタデータクエリを実行すると、新たに作成されたスナップショットが、その保持期間を超過した場合でも、削除されずに保持されることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_practice.history

## データ削除要件にもとづいて削除する

より実際のユースケースについて確認してみましょう。例えば、ある会社で削除したレコードを一定期間保持し、その後データ自体を削除するポリシーがあるとします。ここでは、テストとしてその期間を**5 分**とし、実際にどのような手順で削除が可能か確認します。まずはテーブルを作成し、最初にデータをロードします。

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg
TBLPROPERTIES (
    'history.expire.max-snapshot-age-ms'='300000' -- スナップショット保持期間を 5 min に設定
)

In [ ]:
%%sql
INSERT INTO db.sales_iceberg VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink'),
    ('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery'),
    ('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg

実際にデータを削除まで行ってみましょう。以下のセルでは、2 分ごとにレコードを削除し、`expire_snapshots`を実行します。

In [ ]:
deleted_products = [
    "DRE8DLTFNX0MLCE8DLTFNX0MLC", # tomato juice
    "DR1UNFHET81UNFHET8", # cocoa
    "GRK0L8ZQK0L8ZQ", # broccoli
]

# レコードの削除 (データファイル自体は削除されない)
for deleted_product in deleted_products:
    time.sleep(120) # 2 分ごとにレコードを削除する
    spark.sql(f"DELETE FROM db.sales_iceberg WHERE order_id = '{deleted_product}'")

# データの削除 (データファイルが削除される)
print("Running expire_snapshots...")
spark.sql("""
CALL system.expire_snapshots (
    table => 'db.sales_iceberg'
)
""").show(truncate=False)

削除後のテーブルレコードを確認してみましょう。2 レコード (`espresso`, `nutmeg`)のみ出力されます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg

テーブルの履歴の確認と、各スナップショットに対するタイムトラベルクエリを実行してみましょう。2 スナップショット表示され、最初に 5 レコード書き込まれた際のスナップショットと、削除された `tomato juice`に関するスナップショットが削除されていることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg.history

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg VERSION AS OF <一番古い snapshot_id>
-- tomato juice を削除した後のスナップショットにアクセスできます

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg VERSION AS OF <二番目に古い snapshot_id>
-- tomato juice, cocoa を削除した後のスナップショットにアクセスできます

## ブランチやタグによる詳細なデータの保持管理

本節ではブランチ・タグを利用し、さらに詳細なデータ保持管理を行います。具体的には以下の内容を実施します:

* `main` ブランチに加え `dev` ブランチを作成
* `main` ブランチのスナップショット保持期間を 10 分に設定 (テストのため短く設定しています)
* `main` ブランチを除く、ブランチ全体の参照期間を 8 分に設定
* `main` ブランチにおけるテーブルの初期状態に `initial_state` タグを付与する。このタグの期間を 1 時間に設定
* `dev` ブランチのスナップショット保持期間を 4 分に設定

まずは事前準備としてテーブルを作成し、3 レコードを追加します。

In [ ]:
%%sql
CREATE OR REPLACE TABLE db.sales_iceberg_adv (
    product_name string,
    price decimal(10, 2),
    customer_id bigint,
    order_id string,
    datetime timestamp,
    category string) 
USING iceberg
TBLPROPERTIES (
    'history.expire.max-snapshot-age-ms'='600000', -- 10 分に設定
    'history.expire.max-ref-age-ms' = '480000' -- ブランチの参照期間を 8 分に設定
)

In [ ]:
%%sql
INSERT INTO db.sales_iceberg_adv VALUES
    ('tomato juice', 2.00, 1698, 'DRE8DLTFNX0MLCE8DLTFNX0MLC', TIMESTAMP '2023-07-18T02:20:58Z', 'drink'),
    ('cocoa', 2.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2024-08-26T11:36:48Z', 'drink'),
    ('espresso', 2.00, 1037, 'DRBFZUJWPZ9SRABFZUJWPZ9SRA', TIMESTAMP '2024-04-19T12:17:22Z', 'drink')

この時点でのテーブルレコードを確認します。3 レコード出力されます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv

### タグの付与
初期のテーブル時点を削除しないように、保持するため、タグを付与します。このタグは 1 時間保持するよう設定します。

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg_adv CREATE TAG initial_state RETAIN 1 HOURS

### さらにテーブルにデータを書き込む
さらにこの後テーブルにデータが一定間隔で書き込まれたことをシミュレートするため、以下のセルを実行してください。

In [ ]:
records = [
    "('broccoli', 1.00, 3092, 'GRK0L8ZQK0L8ZQ', TIMESTAMP '2023-03-22T18:48:04Z', 'grocery')",
    "('nutmeg', 1.00, 3512, 'GR15U0LKA15U0LKA', TIMESTAMP '2024-02-27T15:13:31Z', 'grocery')",
]

for record in records:
    time.sleep(150) # 2 分 30 秒ごとにテーブルに書き込む
    spark.sql(f"INSERT INTO db.sales_iceberg_adv VALUES {record}")

この時点でのテーブルレコードとスナップショット履歴を確認しておきましょう。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.history

この時点における Iceberg テーブルのスナップショットと各期限を確認しておきましょう。

```
[main branch: スナップショット保持期間 10 分]
snap_0 (3 レコード追加, initial_state TAG 付与) 
         ↓
snap_1 (1 レコード - broccoli 追加) 
         ↓
snap_2 (1 レコード - nutmeg 追加)
```

### ブランチの作成

`dev`ブランチを作成します。ここでは`dev`ブランチにおけるスナップショットの保持期間を 4 分に設定しています。

In [ ]:
%%sql
ALTER TABLE db.sales_iceberg_adv CREATE BRANCH dev WITH SNAPSHOT RETENTION 4 MINUTES

`refs`メタデータテーブルに対してクエリを実行しブランチ・タグ一覧を取得します。`initial_state`タグが付与されていることと、最初のスナップショットに付与されていることと、`dev`ブランチが作成されていることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.refs

### 各ブランチにおけるデータ保持期間に基づくスナップショット削除

作成した`dev`ブランチにデータを書き込むことを想定します。データは 3 分ごとに書かれ、書き込みが完了した後に`expire_snapshots`を実行し、スナップショットを削除します。

In [ ]:
records = [
    "('mocha', 4.00, 1652, 'DR1UNFHET81UNFHET8', TIMESTAMP '2013-11-26T12:49:43Z', 'drink')",
    "('egg', 1.00, 3176, 'GRVQARCD6COVQARCD6CO', TIMESTAMP '2025-02-10 11:15:31', 'grocery')",
]

for record in records:
    spark.sql(f"INSERT INTO db.sales_iceberg_adv.branch_dev VALUES {record}")
    time.sleep(180) # 3 分ごとにテーブルに書き込む

print("Running expire_snapshots...")
spark.sql("""
CALL system.expire_snapshots (
    table => 'db.sales_iceberg_adv'
)
""").show(truncate=False)

この時点におけるスナップショット一覧を取得します。ブランチも含めたスナップショット一覧は `snapshots` メタデータテーブルに対してクエリを実行することで取得できます。以下のスナップショットが保持されます:
* `main` ブランチで最初に 3 レコードを追加した際のスナップショット (`initial_state` タグを付与しており、保持期間を 1 時間に設定しているため削除されません)
* `main` ブランチの最新とその 1 つ前のスナップショット (10 分間の保持期間内であるため削除されません)
* `dev` ブランチでで最新のスナップショット (4 分間の保持期間内であるため削除されません)

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.snapshots

`refs` メタデータテーブルにクエリし、各ブランチの最新の snapshot_id と、タグが付与されている snapshot_id を取得します (合計で 3 snapshot_id 確認できます)。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.refs

最後にもう一度 `expire_snapshots` を実行します。以下では、事前に 2 分ほど待機時間を設けています。これはブランチ参照期間として設定した 8 分を超過した際に `refs` メタデータテーブルから、作成して `dev` ブランチへの参照がなくなっていることを確認するためです。

In [ ]:
time.sleep(120) # ブランチ参照期間 8 分を超過させるため、2 分間おきます
spark.sql("""
CALL system.expire_snapshots (
    table => 'db.sales_iceberg_adv'
)
""").show(truncate=False)

最後に現時点でのスナップショットの状態を確認します。タグが付与された最初のテーブルの状態を表すスナップショットと、`main` ブランチの最新のスナップショットが保持されていることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.snapshots

`refs` メタデータテーブルにクエリし、各ブランチの最新の snapshot_id と、タグが付与されている snapshot_id も取得してみましょう。`dev` ブランチが削除されていることと、タグが付与されたスナップショットが保持されていることを確認できます。

In [ ]:
%%sql
SELECT * FROM db.sales_iceberg_adv.refs